# Robotics, Vision & Control 3e: for Python
## Chapter 11: Images & Image processing

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    COLAB = True
except:
    COLAB = False
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
from math import pi
from os import getenv
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})
np.random.seed(0)
from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import SE2, Twist2


# 11.1 Obtaining an Image
## 11.1.1 Images from Files


In [ ]:
street, _ = iread("street.png");

In [ ]:
street

In [ ]:
street.shape

In [ ]:
street[400, 200]

In [ ]:
idisp(street);

In [ ]:
flowers, _ = iread("flowers8.png")

In [ ]:
flowers.dtype

In [ ]:
idisp(flowers);

In [ ]:
flowers.shape

In [ ]:
idisp(flowers[:, :, 0]);

In [ ]:
pix = flowers[276, 318, :]

In [ ]:
street = Image.Read("street.png")

In [ ]:
street.shape

In [ ]:
street.disp();

In [ ]:
street.min()
street.max()

In [ ]:
street.stats()

In [ ]:
img = street.image;
type(img)

In [ ]:
street.image[400, 200]

In [ ]:
subimage = street.image[100:200, 200:300];
type(subimage)

In [ ]:
subimage = street[100:200, 200:300];
type(subimage)

In [ ]:
flowers = Image.Read("flowers8.png")

In [ ]:
flowers.stats()

In [ ]:
flowers.image[276, 318, :]

In [ ]:
flowers.image[:, :, 0];

In [ ]:
flowers[:, :, 0].disp();

In [ ]:
flowers.plane(0).disp();

In [ ]:
flowers.plane("R").disp();

In [ ]:
flowers.red().disp();

In [ ]:
flowers.plane("B:R:G")

In [ ]:
flowers.plane("B:G:R").disp();

In [ ]:
church = Image.Read("church.jpg");
church.metadata()

## 11.1.2 Images from File Sequences


In [ ]:
images = ImageCollection("seq/*.png");
len(images)

In [ ]:
images[3]

In [ ]:
for image in images:
  image.disp()  # do some operation

In [ ]:
# this may fail unless you have installed these extra image zip files
# see https://github.com/petercorke/machinevision-toolbox-python/tree/master/mvtb-data#install-big-image-files
images = ZipArchive("bridge-l.zip", "*.pgm");
len(images)

## 11.1.3 Images from an Attached Camera


In [ ]:
if COLAB:
    print("there is no camera available in the Colab environment")
else:
    camera = VideoCamera(1)

In [ ]:
image = camera.grab()
image = camera.grab()
camera.release()

In [ ]:
image.disp()

## 11.1.4 Images from a Video File


In [ ]:
video = VideoFile("traffic_sequence.mp4")

In [ ]:
video.shape

Note that from inside Jupyter the images are not shown as an animation, they are displayed as separate frames.

In [ ]:
# for frame in video:
#   frame.disp(reuse=True, fps=video.fps) # display frame in the same axes

## 11.1.5 Images from the Web


In [ ]:
dartmouth = WebCam("https://webcam.dartmouth.edu/webcam/image.jpg");

In [ ]:
dartmouth.grab().disp();

## 11.1.6 Images from Space


In [ ]:
if COLAB:
    # you must provide a Google API key, a 39 character string
    world = EarthView(key="API key");
else:
    # you're Google API key must be provided as the envariable GOOGLE_KEY
    world = EarthView()

In [ ]:
world.grab(-27.475722, 153.0285, 17).disp();

In [ ]:
world.grab(-27.475722,153.0285, 15, type="map").disp();

In [ ]:
world.grab(-27.475722,153.0285, 15, type="roads").disp();

## 11.1.7 Images from Code


In [ ]:
image = Image.Ramp(cycles=2, size=500, dir="x");
image = Image.Sin(cycles=5, size=500, dir="y");
image = Image.Squares(number=5, size=500);
image = Image.Circles(number=2, size=500);

In [ ]:
canvas = Image.Zeros(1000, 1000, dtype="uint8")

In [ ]:
canvas.draw_box(lt=(100, 100), wh=(150, 150), color=100, thickness=-1);
canvas.draw_box(lt=(300, 300), wh=(80, 80), color=150, thickness=-1);

In [ ]:
canvas.draw_circle((600, 600), 120, color=200, thickness=-1)

In [ ]:
canvas.draw_line((100, 100), (800, 800), color=250, thickness=8)

In [ ]:
canvas.disp();

In [ ]:
#  11.2 Pixel Value Distribution
#
plt.close('all')
church = Image.Read("church.png", mono=True)

In [ ]:
church.min()
church.max()
church.mean()
church.median()
church.std()

In [ ]:
church.stats()

In [ ]:
h = church.hist()
h.plot();

In [ ]:
h.plot("ncdf", color="blue")

In [ ]:
x = h.peaks();
x.shape

In [ ]:
x = h.peaks(scale=25)

# 11.3 Monadic Operations


In [ ]:
church_float = church.to("float")
church.max()
church_float.max()

In [ ]:
church_float.to("uint8")

In [ ]:
street_float = Image.Read("street.png", dtype="float")

In [ ]:
gray = flowers.mono()

In [ ]:
color = gray.colorize()

In [ ]:
color = gray.colorize((1, 0, 0))

In [ ]:
bright = (church >= 180)

In [ ]:
bright.disp();

In [ ]:
church.stretch().stats()

In [ ]:
im = church.normhist();

In [ ]:
im = church.gamma_decode(1 / 2.2);

In [ ]:
im = church.gamma_decode("sRGB");

In [ ]:
(church // 64).disp();

In [ ]:
church.apply(lambda x: x // 64, vectorize=True).disp()

In [ ]:
church.npixels

In [ ]:
church.apply(lambda x: x // 64).disp();

In [ ]:
lut = [x // 64 for x in range(256)];  # create posterization lookup table
church.LUT(lut).disp();

# 11.4 Dyadic Operations


In [ ]:
church / 2       # new Image with all pixel values halved
church + 20      # new Image with all pixel values increased by 20
church - church  # new Image with all pixel values equal to 0

In [ ]:
church.apply2(church, lambda x, y: x - y, vectorize=True).disp()

In [ ]:
church.apply2(church, lambda x, y: x - y).disp();

In [ ]:
a = np.uint8(100)
b = np.uint8(200)

In [ ]:
a + b
a - b

In [ ]:
-a

In [ ]:
a / b

## 11.4.1 Applications
### 11.4.1.1 Application: Chroma Keying


In [ ]:
foreground = Image.Read("greenscreen.png", dtype="float")

In [ ]:
cc = foreground.gamma_decode("sRGB").chromaticity()

In [ ]:
cc.plane("g").hist().plot()

In [ ]:
mask = cc.plane("g") < 0.45;
mask.disp();

In [ ]:
(foreground * mask).disp();

In [ ]:
background = Image.Read("road.png", dtype="float").samesize(foreground)

In [ ]:
(background * ~mask).disp();

In [ ]:
composite = foreground * mask  + background * ~mask;
composite.disp();

In [ ]:
background.choose(foreground, mask).disp();

### 11.4.1.2 Application: Motion detection


In [ ]:
video = VideoFile("traffic_sequence.mp4", mono=True, dtype="float")

In [ ]:
sigma = 0.02;
background = None
for im in video:
  if background is None:
    background = im  # first frame only
  else:
    d = im - background
    background += d.clip(-sigma, sigma)
  background.disp(fps=video.fps, block=None)

# 11.5 Spatial Operations
## 11.5.1 Linear Spatial Filtering


O = I.convolve(K);

In [ ]:
# ### 11.5.1.1 Image Smoothing
#
plt.close('all')

In [ ]:
K = Image.Constant(21, 21, value=1/21**2);
K.shape

In [ ]:
mona = Image.Read("monalisa.png", mono=True, dtype="float")
mona.convolve(K).disp();

In [ ]:
K = Kernel.Box(h=10);

In [ ]:
K = Kernel.Gauss(sigma=5);
mona.convolve(K).disp();

In [ ]:
K.shape

In [ ]:
mona.smooth(sigma=5).disp();

In [ ]:
idisp(K);

In [ ]:
span = np.arange(-15, 15 + 1);
X, Y = np.meshgrid(span, span)
plt.subplot(projection="3d").plot_surface(X, Y, K);

In [ ]:
K = Kernel.Circle(radius=8, h=15);

In [ ]:
K=Kernel.Box(h=1)
np.linalg.matrix_rank(K)

In [ ]:
U, s, Vh  = np.linalg.svd(K, full_matrices=True)
Kh = s[0] * U[:, 0]  # 1D horizontal kernel
Kv = Vh[0, :]        # 1D vertical kernel

In [ ]:
np.outer(Kh, Kv) 

### 11.5.1.2 Border Extrapolation
### 11.5.1.3 Edge Detection


In [ ]:
plt.clf()

In [ ]:
castle = Image.Read("castle.png", mono=True, dtype="float")

In [ ]:
profile = castle.image[360, :];
profile.shape

In [ ]:
plt.plot(profile);

In [ ]:
plt.plot(np.diff(profile));

In [ ]:
K = [0.5, 0, -0.5];
castle.convolve(K).disp(colormap="signed");

In [ ]:
Du = Kernel.Sobel()

In [ ]:
castle.convolve(Du).disp(colormap="signed");

In [ ]:
castle.convolve(Du.T).disp(colormap="signed");

In [ ]:
from scipy.signal import convolve2d
Gu = convolve2d(Du, Kernel.Gauss(sigma=1));
Gu.shape

In [ ]:
Xu = castle.convolve(Kernel.DGauss(sigma=2));
Xv = castle.convolve(Kernel.DGauss(sigma=2).T);

In [ ]:
m = (Xu ** 2 + Xv ** 2).sqrt()

In [ ]:
th = Xv.apply2(Xu, np.arctan2);  # arctan2(Xv, Xu)

In [ ]:
plt.quiver(castle.uspan(20), castle.vspan(20),
  Xu.image[::20, ::20], Xv.image[::20, ::20], scale=10);

In [ ]:
Xu, Xv = castle.gradients(Kernel.DGauss(sigma=2))

In [ ]:
edges = castle.canny()

In [ ]:
L = Kernel.Laplace()

In [ ]:
lap = castle.convolve(Kernel.LoG(sigma=2));

In [ ]:
profile = lap.image[360, 570:601];
plt.plot(np.arange(570, 601), profile, "-o");

In [ ]:
zc = lap.zerocross();

## 11.5.2 Template Matching


In [ ]:
mona = Image.Read("monalisa.png", mono=True, dtype="float");
A = mona.roi([170, 220, 245, 295]);

In [ ]:
B = A
A.sad(B)
A.ssd(B)
A.ncc(B)

In [ ]:
B = 0.9 * A
A.sad(B)
A.ssd(B)

In [ ]:
A.ncc(B)

In [ ]:
B = A + 0.1
A.sad(B)
A.ssd(B)
A.ncc(B)

In [ ]:
A.zsad(B)
A.zssd(B)
A.zncc(B)

In [ ]:
B = 0.9 * A + 0.1
A.zncc(B)

### 11.5.2.1 Application: Finding Wally


In [ ]:
crowd = Image.Read("wheres-wally.png", mono=True, dtype="float")
crowd.disp();

In [ ]:
T = Image.Read("wally.png", mono=True, dtype="float")
T.disp();

In [ ]:
sim = crowd.similarity(T, "zncc")

In [ ]:
sim.disp(colormap="signed", colorbar=True);

In [ ]:
maxima, location = sim.peak2d(scale=2, npeaks=5)
maxima

In [ ]:
location

In [ ]:
crowd.disp(block=None);
plot_circle(centre=location, radius=20, color="k");
plot_point(location, color="none", marker="none", text="  #{}");

### 11.5.2.2 Nonparameteric Local Transforms
## 11.5.3 Nonlinear Operations


In [ ]:
out = mona.window(np.var, h=3);

In [ ]:
mx = mona.rank(rank=0, h=2);

In [ ]:
med = mona.rank(rank=11, h=2);

In [ ]:
spotty = mona.copy()
pixels = spotty.view1d();  # create a NumPy 1D view
npix = mona.npixels    # total number of pixels
k = np.random.choice(npix, 10_000, replace=True);  # choose 10,000 unique pixels
pixels[k[:5_000]] = 0  # set half of them to zero
pixels[k[5_000:]] = 1  # set half of them to one
spotty.disp();

In [ ]:
spotty.rank(rank=4, h=1).disp();

In [ ]:
M = np.full((3, 3), True);
M[1, 1] = False
M

In [ ]:
max_neighbors = mona.rank(rank=0, footprint=M);

In [ ]:
(mona > max_neighbors).disp();

# 11.6 Mathematical Morphology


In [ ]:
im = Image.Read("eg-morph1.png")
im.disp();

In [ ]:
S = np.ones((5, 5));

In [ ]:
e1 = im.morph(S, op="min")

morphdemo(im, S, op="min")

In [ ]:
d1 = e1.morph(S, op="max")

In [ ]:
out = im.erode(S);
out = im.dilate(S);

## 11.6.1 Noise Removal


In [ ]:
objects = Image.Read("segmentation.png")
objects.disp();

In [ ]:
S_circle = Kernel.Circle(3)

In [ ]:
closed = objects.close(S_circle);

In [ ]:
clean = closed.open(S_circle);

In [ ]:
objects.open(S_circle).close(S_circle).disp();

## 11.6.2 Boundary Detection


In [ ]:
eroded = clean.erode(S_circle)

In [ ]:
edge = clean - eroded
edge.disp();

## 11.6.3 Hit or Miss Transform


out = image.hitormiss(S);

In [ ]:
skeleton = clean.thin();

In [ ]:
ends = skeleton.endpoint()

In [ ]:
joins = skeleton.triplepoint();

## 11.6.4 Distance Transform


In [ ]:
im = Image.Squares(1, size=256).rotate(0.3).canny()

In [ ]:
dx = im.distance_transform(norm="L2")

# 11.7 Shape Changing
## 11.7.1 Cropping


In [ ]:
plt.close('all')

In [ ]:
mona.disp();

Select a region of interest on the image, left click then drag
eyes, roi = mona.roi();
eyes

roi

In [ ]:
smile = mona.roi([265, 342, 264, 286])
smile.disp();

## 11.7.2 Image Resizing


In [ ]:
roof = Image.Read("roof.png", mono=True)

In [ ]:
roof[::7, ::7].disp();

In [ ]:
roof.smooth(sigma=3)[::7, ::7].disp();

In [ ]:
smaller = roof.scale(1/7, sigma=3)

In [ ]:
smaller.replicate(7).disp();

## 11.7.3 Image Pyramids


In [ ]:
pyramid = mona.pyramid()
len(pyramid)

## 11.7.4 Image Warping


In [ ]:
Up, Vp = Image.meshgrid(width=500, height=500)

In [ ]:
U = 4 * (Up - 100);
V = 4 * (Vp - 200);

In [ ]:
p = (300, 200);  # (v, u)
(U[p], V[p])

In [ ]:
p = (100, 200);  # (v, u)
(U[p], V[p])

In [ ]:
mona.warp(U, V).disp();

In [ ]:
M = np.diag([0.25, 0.25, 1]) * SE2(100, 200) 

In [ ]:
out = mona.warp_affine(M, bgcolor=np.nan);
out.disp(badcolor="r");

In [ ]:
S = Twist2.UnitRevolute(mona.centre)

In [ ]:
M = S.exp(pi / 6)
out = mona.warp_affine(M, bgcolor=np.nan)

In [ ]:
twisted_mona = mona.rotate(pi/6);
twisted_mona.disp()